# Import Libraries

In [1]:
import gurobipy as grb
import pandas as pd
from gurobipy import GRB
import numpy as np

# Model

In [2]:
 # Create a model
model = grb.Model("Model")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-10


# Get data from json

In [3]:
# get the  data from the json file small.json
import json
with open('small.json') as f:
    data = json.load(f)
# data

## Constants from data

In [4]:
horizon= data['horizon']
qualifications=data['qualifications']
staff_names=[i['name'] for i in data['staff']]
staff_qualifications=[i['qualifications'] for i in data['staff']]
job_list=[i['name'] for i in data['jobs']]
jour_list=[i for i in range(1,horizon+1)]
job_penality=[i['daily_penalty'] for i in data['jobs']] 
job_qualifications= [i['working_days_per_qualification'] for i in data['jobs']]
job_gain=[i['gain'] for i in data['jobs']]
due_dates=[i['due_date'] for i in data['jobs']]
vacation_staff=[i['vacations'] for i in data['staff']]

# Model's parameters

#### Objective 1
H : days

Q : qualifications

S : staff

J : jobs

qi : qualifications of i

qj : qualifications of j

vi : vacation days of i

njk : required work load for j with k

cj : daily penalty of j

gj : gain of j

dj : due date of j

yj : is j completely done

lj : delay in days for j

ej : end date of j

pi_j_k_t : work is done by i with k for j on t

#### Objective 2
aij : j is assigned to i

ni : number of jobs i works on

max_i : max of ni

### Objective 3
sj : start date for j

nj : length in days for j

max_j : maximum of nj

In [5]:
H = [i for i in range(1,horizon+1)]
Q = qualifications
S = staff_names
J = job_list
# Parameters for each staff

qi = {}
for i in range(len(staff_names)) :
    k = staff_names[i]
    qi[k] = []
    for c in qualifications:
        if c in data['staff'][i]['qualifications']:
            qi[k].append(c)
vi={}
for ind,i in enumerate(staff_names):
    vi[i] = []
    for j in jour_list:
        if j in vacation_staff[ind]:
            vi[i].append(j)
#vi=staff_in_vacation.to_dict(orient='index')

# Parameters for each job
qj = dict()
for ind, j in enumerate(job_list):
    qj[j] = list(job_qualifications[ind].keys())
njk = dict(zip(job_list,job_qualifications))
cj=job_penality_dict=dict(zip(job_list,job_penality))
gj=job_gain_dict=dict(zip(job_list,job_gain))
dj = dict(zip(job_list,due_dates))


# Decision variables

In [6]:
# Objective 1
pijkt = model.addVars(staff_names,job_list,qualifications,jour_list, vtype=GRB.BINARY, name="pijkt")

yj = model.addVars(job_list, vtype=GRB.BINARY, name="yj")
lj = model.addVars(job_list, vtype=GRB.INTEGER, name="lj")
ej = model.addVars(job_list, vtype=GRB.INTEGER, name="ej")

# Objective 2
max_i = model.addVar(vtype=GRB.INTEGER, name="max_i")
ni = model.addVars(S, vtype=GRB.INTEGER, name="ni")
aij = model.addVars(S, J, vtype=GRB.BINARY, name="aij")

# Objective 3
sj = model.addVars(J, vtype=GRB.INTEGER, name="sj")
nj = model.addVars(J, vtype=GRB.INTEGER, name="nj")
max_j = model.addVar(vtype=GRB.INTEGER, name="max_j")

# Constraints

### Objective 1

In [7]:
constr1 = {f'constr1{i}_{t}':model.addConstr( grb.quicksum(pijkt[i,j,k,t] for j in J for k in Q) <= 1 , name=f"constr1{i}_{t}")
  for i in S
  for t in H}

constr2 = {f'constr2{i}_{t}':model.addConstr( grb.quicksum(pijkt[i,j,k,t] for j in J for k in Q) == 0 , name=f"constr2{i}_{t}")
  for i in S
  for t in vi[i]}

constr3 = {f'constr3{i}_{j}_{k}_{t}':model.addConstr( pijkt[i,j,k,t] == 0 , name=f"constr3{i}_{j}_{k}_{t}")
  for i in S
  for j in J
  for k in Q if k not in qj[j] or k not in qi[i]
  for t in H}

constr4 = {f'constr4{j}_{k}': model.addConstr( yj[j]*njk[j][k] <= grb.quicksum(pijkt[i,j,k,t] for i in S for t in H) , name=f"constr4{j}_{k}")
  for j in J
  for k in qj[j]}

constr5 = {f'constr5{j}_{k}': model.addConstr( grb.quicksum(pijkt[i,j,k,t] for i in S for t in H) <= njk[j][k] , name=f"constr5{j}_{k}")
  for j in J
  for k in qj[j]}

constr6 = {f'constr6{i}_{j}_{k}_{t}':model.addConstr( pijkt[i,j,k,t]*t <= ej[j] , name=f"constr6{i}_{j}_{k}_{t}")
  for i in S
  for j in J
  for k in Q
  for t in H}

constr7 = {f'constr7{j}' : model.addConstr( ej[j] - dj[j] <= lj[j] , name =f"constr7{j}")
  for j in J}

constr8 = {f'constr8{j}' : model.addConstr( 1 <= ej[j] , name =f"constr8{j}")
  for j in J}

constr9 = {f'constr9{j}' : model.addConstr( ej[j] <= H[-1] , name =f"constr9{j}")
  for j in J}

### Objective 2

In [8]:
# assigned jobs to a staff member i is aij[i, j]
constr10 = {f'constr10{i}_{j}_{k}_{t}' : model.addConstr( pijkt[i,j,k,t] <= aij[i, j] , name = f"constr10{i}_{j}_{k}_{t}")
    for i in S
    for j in J
    for k in Q
    for t in H}

# number of jobs assigned to a staff member i is ni[i]
constr11 = {f'constr11{i}' : model.addConstr( grb.quicksum(aij[i, j] for j in J) <= ni[i] , name = f"constr11{i}")
    for i in S}

# max_i max number of jobs assigned to a staff member
# for all staff i, number of jobs assigned to i is less than or equal to max_i
constr12 = {f'constr12{i}' : model.addConstr( ni[i] <= max_i , name = f"constr12{i}")
    for i in S}

### Objective 3

In [9]:
# start date is at least 1
constr14 = {f'constr14_{j}' : model.addConstr( 1 <= sj[j] , name = f"constr14_{j}")
    for j in J}

# start date of j is sj[j]
constr15 = {f'constr15{i}_{j}_{k}_{t}' : model.addConstr( sj[j] <= t*pijkt[i,j,k,t] + H[-1]*(1-pijkt[i,j,k,t]) , name = f"constr15{i}_{j}_{k}_{t}")
    for i in S
    for j in J
    for k in Q
    for t in H}

# length of a job j is nj[j]
constr16 = {f'constr16{j}' : model.addConstr( ej[j] + 1 - sj[j] <= nj[j] , name = f"constr16{j}")
    for j in J}

# max_j max of nj[j]
constr17 = {f'constr17{j}' : model.addConstr( nj[j] <= max_j , name = f"constr17{j}")
    for j in J}

# Objectives

In [19]:
# Objective 1
# model.setObjective( grb.quicksum((gj[j]*yj[j] - lj[j]*cj[j]) for j in J) , GRB.MAXIMIZE)

# Objective 1 & 2
# model.setObjective( grb.quicksum((gj[j]*yj[j] - lj[j]*cj[j]) for j in J) - max_i , GRB.MAXIMIZE)

# Objective 1 & 2 & 3
# model.setObjective( grb.quicksum((gj[j]*yj[j] - lj[j]*cj[j]) for j in J) - max_i - max_j , GRB.MAXIMIZE)

# setObjectiveN
model.ModelSense = GRB.MAXIMIZE
model.setObjectiveN( grb.quicksum((gj[j]*yj[j] - lj[j]*cj[j]) for j in J) , 0, 3)
model.setObjectiveN( - max_i , 1, 1)
model.setObjectiveN( - max_j , 2, 1)

# Paramétrage (mode mute)
model.params.outputflag = 1
# Résolution du PL
model.optimize()

Set parameter OutputFlag to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 873 rows, 270 columns and 2069 nonzeros
Model fingerprint: 0xe5bdd0c1
Variable types: 0 continuous, 270 integer (245 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives (2 combined) ...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 703 rows and 135 columns
Presolve time: 0.00s
Presolved: 170 rows and 135 columns
---------------------------------------

In [11]:
#model.computeIIS()
model.write("model.lp")

In [23]:
# Query number of multiple objectives, and number of solutions
nSolutions  = model.SolCount
nObjectives = model.NumObj
print('Problem has', nObjectives, 'objectives')
print('Gurobi found', nSolutions, 'solutions')

Problem has 3 objectives
Gurobi found 9 solutions


In [26]:
solutions = []
for s in range(nSolutions):
  # Set which solution we will query from now on
    model.params.SolutionNumber = s

  # Print objective value of this solution in each objective
    print('Solution', s, ':', end='')
    for o in range(nObjectives):
        # Set which objective we will query
        model.params.ObjNumber = o
        # Query the o-th objective value
        print(' ',model.ObjNVal, end='')
    print('\n')

Solution 0 :  65.0  -3.0  -2.0

Solution 1 :  65.0  -3.0  -3.0

Solution 2 :  65.0  -4.0  -3.0

Solution 3 :  65.0  -4.0  -4.0

Solution 4 :  65.0  -5.0  -5.0

Solution 5 :  59.0  -5.0  -5.0

Solution 6 :  57.0  -5.0  -5.0

Solution 7 :  20.0  -5.0  -5.0

Solution 8 :  0.0  -2000000000.0  -2000000000.0



In [52]:
# Choix de la solution et de la fonction objective
model.params.SolutionNumber = 5
model.params.ObjNumber = 0
model.ObjNVal

59.0

In [48]:

values= []
for k, v in pijkt.items():
    values.append(v.Xn)

In [49]:
res = pd.DataFrame(list(pijkt.keys()))
res[4] = values
result = res[res[4] == 1]
result

,0,1,2,3,4
2,Olivia,Job1,A,3,1.0
43,Olivia,Job3,C,4,1.0
50,Olivia,Job4,B,1,1.0
51,Olivia,Job4,B,2,1.0
74,Olivia,Job5,C,5,1.0
84,Liam,Job1,B,5,1.0
91,Liam,Job2,A,2,1.0
97,Liam,Job2,B,3,1.0
108,Liam,Job3,A,4,1.0
160,Emma,Job1,C,1,1.0


## Result Table

In [50]:
def color_table(x):
    if pd.isna(x):
        return "background-color: white"
    else:
        if "Job1" in x :
            return "background-color: blue"
        elif "Job2" in x:
            return "background-color: pink"
        elif "Job3" in x:
            return "background-color: orange"
        elif "Job4" in x:
           return "background-color: yellow"
        elif "Job5" in x:
           return "background-color: green"
        else:
           return  "background-color: red"

In [51]:
df = pd.DataFrame(columns = [i for i in range(1,6)], index = staff_names)

for ind, val in result.iterrows():
    col = val[3]
    row = val[0]
    v = val[1] + " " + val[2]
    df.loc[row,col] = v
    # vacation
    for staff in staff_names:
        for day in jour_list:
            if day in vi[staff]:
                df.loc[staff,day] = 'X'
df.style.applymap(color_table)

,1,2,3,4,5
Olivia,Job4 B,Job4 B,Job1 A,Job3 C,Job5 C
Liam,X,Job2 A,Job2 B,Job3 A,Job1 B
Emma,Job1 C,X,Job4 C,Job3 C,Job5 C


In [17]:

# start date
sj

{'Job1': <gurobi.Var sj[Job1] (value 1.0)>,
 'Job2': <gurobi.Var sj[Job2] (value 2.0)>,
 'Job3': <gurobi.Var sj[Job3] (value 3.0)>,
 'Job4': <gurobi.Var sj[Job4] (value 2.0)>,
 'Job5': <gurobi.Var sj[Job5] (value 4.0)>}

In [794]:
# end date
ej

{'Job1': <gurobi.Var ej[Job1] (value 2.0)>,
 'Job2': <gurobi.Var ej[Job2] (value 1.0)>,
 'Job3': <gurobi.Var ej[Job3] (value 4.0)>,
 'Job4': <gurobi.Var ej[Job4] (value 3.0)>,
 'Job5': <gurobi.Var ej[Job5] (value 5.0)>}

In [795]:
# length
nj

{'Job1': <gurobi.Var nj[Job1] (value 2.0)>,
 'Job2': <gurobi.Var nj[Job2] (value 2.0)>,
 'Job3': <gurobi.Var nj[Job3] (value 2.0)>,
 'Job4': <gurobi.Var nj[Job4] (value 2.0)>,
 'Job5': <gurobi.Var nj[Job5] (value 2.0)>}